# Temporal Difference Learning for 3D Tic Tac Toe

This notebook contains the implementation of a Temporal Difference (TD) learning model using a Deep Q-Network (DQN) for playing 3D 4x4x4 Tic Tac Toe. The implementation is based on the approach outlined in the provided paper.


In [1]:
# Import necessary libraries
import numpy as np
import os
import random

# Setting Directory
os.chdir('C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/reinforcement_learning/Project/')

from python_scripts import state_formulation, utils, algorithm, tictactoe_4x4
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
import torch.nn.init as init
from tqdm.autonotebook import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
class customDotProduct(nn.Module):
    def __init__(self, input_size, output_size, block_size = 4):
        super(customDotProduct, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.block_size = block_size
        # Convert structure_weight to nn.Parameter
        self.structure_weight = torch.zeros((self.output_size, self.input_size))
        self.structure_weight = self.get_block_weights(self.structure_weight, block_size)
        self.structure_weight = nn.ParameterList([nn.Parameter(sw.float()) for sw in self.structure_weight])

    def get_block_weights(self, weight_list, block_size):
        for i in range(0, 304, block_size):
            weight_list[i: i + block_size, i: i + block_size] = init.xavier_normal_(torch.randn(block_size, block_size))
        learnable_blocks = [weight_list[i:i + block_size, i:i + block_size] for i in range(0, weight_list.shape[0], block_size)]
        updated = [block for block in learnable_blocks]
        return updated
    
    def forward(self, feature_map):
        self.feature_map = [fm.float() for fm in feature_map]
        # Calculate dot products and concatenate along dim = 1
        concatenated_products = torch.cat([torch.matmul(fm.unsqueeze(0), sw) for fm, sw in zip(self.feature_map, self.structure_weight)], dim = 1)
        return concatenated_products

In [3]:
class StructuredLinear(nn.Module):
    def __init__(self):
        super(StructuredLinear, self).__init__()

    def get_rows(self, input_tensor):
        # Get diagonals (across 2 faces),digonals (across 3 faces) and horizontal and vertical rows
        diag_two_faces = []
        diag_two_faces.extend(
            [torch.diagonal(input_tensor[i, :, :]), torch.diagonal(input_tensor[:, i, :]), torch.diagonal(input_tensor[:, :, i]), 
            torch.diagonal(torch.fliplr(input_tensor)[i, :, :]), torch.diagonal(torch.fliplr(input_tensor)[:, i, :]), torch.diagonal(torch.fliplr(input_tensor)[:, :, i])] 
            for i in range(input_tensor.shape[0]))
        diag_two_faces = [item for sublist in diag_two_faces for item in sublist]
        
        diag_three_faces = []
        diag_three_faces = [[[[input_tensor[i, i, i], input_tensor[3 - i, i, i], input_tensor[i, 3 - i, i], input_tensor[i, i, 3 - i]] 
                            for i in range(4)][k][j] for j in range(4) for k in range(4)][l:l + 4] for l in range(0, 16, 4)]
        diag_three_faces = [torch.tensor([t.item() for t in row]) for row in diag_three_faces]

        horizontal_and_vertical_rows = []
        horizontal_and_vertical_rows.extend([input_tensor[i, j, :], input_tensor[i, :, j], input_tensor[:, i, j]]
                                            for i in range(input_tensor.shape[0]) for j in range(input_tensor.shape[0]))
        horizontal_and_vertical_rows = [item for sublist in horizontal_and_vertical_rows for item in sublist]
        
        return horizontal_and_vertical_rows + diag_two_faces + diag_three_faces

    def forward(self, x):
        rows = self.get_rows(x)
        return rows

class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super(MyNeuralNetwork, self).__init__()
        self.structured_layer = StructuredLinear()
        
        self.custom_operation_layer = customDotProduct(input_size = 304, output_size = 304)

        self.second_layer = nn.Linear(304, 32, bias = False)
        init.xavier_normal_(self.second_layer.weight)

        self.output_layer = nn.Linear(32, 1, bias = False)
        self.act = nn.Tanh()

    def forward(self, x):
        x = self.structured_layer(x)
        x = self.custom_operation_layer(x)
        x = self.act(x) # --> Tanh
        x = self.second_layer(x)
        x = self.act(x)
        x = self.output_layer(x)
        return x
    
# Example usage
# If using a pre-trained model (see report) load the weights like
model = MyNeuralNetwork()
state_dict_runs = torch.load('C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/reinforcement_learning/Project/Phase_3_3D_Tic_Tac_Toe/models/ep_6600.pth', map_location = torch.device(device))
model.load_state_dict(state_dict_runs)

<All keys matched successfully>

In [4]:
summary(model, input_size = [4, 4, 4], col_names = ['input_size', 'output_size', 'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MyNeuralNetwork                          [4, 4, 4]                 [1, 1]                    --
├─StructuredLinear: 1-1                  [4, 4, 4]                 [4]                       --
├─customDotProduct: 1-2                  [4]                       [1, 304]                  1,216
├─Tanh: 1-3                              [1, 304]                  [1, 304]                  --
├─Linear: 1-4                            [1, 304]                  [1, 32]                   9,728
├─Tanh: 1-5                              [1, 32]                   [1, 32]                   --
├─Linear: 1-6                            [1, 32]                   [1, 1]                    32
Total params: 10,976
Trainable params: 10,976
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.04
Estimated Total Size (MB): 0

In [5]:
# Training parameters
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_function = nn.SmoothL1Loss()
EPSILON = 0.1
GAMMA = 0.9

In [8]:
def e_greedy(value_dict):
    if np.random.random() > EPSILON:
        return max(value_dict, key = lambda k: value_dict[k])
    else:
        return random.choice(list(value_dict.items()))[0]

def func_modify(afterstate): # 3d list to 1d tensor
    return torch.tensor([[[1 if cell == "X" else -1 if cell == "O" else 0 for cell in row] for row in layer] for layer in afterstate])

# Initlly agent trained by random actions of other player
def benchmark_policy_for_player2(action_list):
    return np.random.choice(action_list)

def get_coordinates(position):
    x = int((position % 16) % 4)
    y = int((position % 16) / 4)
    z = int(position / 16)

    return x, y, z

# After 5000 or so episodes we resorted to the model itself for the other player's moves. In that case use the 
# defined model weights for it
# For player 2 policy - change ur directory for 5200 here
# model2 = MyNeuralNetwork()
# state_load = torch.load('C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/reinforcement_learning/Project/Phase_3_3D_Tic_Tac_Toe/models/ep_5000.pth', map_location = torch.device(device))
# model2.load_state_dict(state_load)

def policy_player2(model, action_space, observation):
    value_dict = {}
    for action in action_space:
        x, y, z = get_coordinates(action)
        copy_tensor = func_modify(observation).detach().clone()
        copy_tensor[x, y, z] = -1
        value_dict[action] = model.forward(copy_tensor.view(4, 4, 4))
    action = max(value_dict, key = lambda k: value_dict[k])
    return action

def train_td_model(model, num_episodes):
    model.train()
    overall_loss = []
    for episode in tqdm(range(num_episodes)):
        env = tictactoe_4x4.TicTacToe4x4x4()

        terminated = False
        current_state = torch.zeros((64), device = device)
        reward = 0
        player_turn = "X"
        prev_afterstate = None
        time_idx = 0
        loss_list = []

        while not terminated:
            time_idx += 1
            # Get actions space
            action_space = env.get_action_space()
            value_dict = {}
            for action in action_space:
                copy_tensor = current_state.detach().clone()
                copy_tensor[action] = 1
                value_dict[action] = model.forward(copy_tensor.view(4, 4, 4))

            # Here we choose action based on epsilon greedy
            action = e_greedy(value_dict)

            current_state, reward, terminated, player_turn = env.step(action) # afterstate

            if time_idx != 1:
                v_new = reward + (GAMMA * model.forward(func_modify(current_state).to(device)))
                v = model.forward(func_modify(prev_afterstate).to(device))
                loss = loss_function(v, v_new)
                loss_list.append(loss.item())
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            prev_afterstate = current_state
            # Player 2 -- use random or model itself
            player_2_action_space = env.get_action_space()
            player_2_move = benchmark_policy_for_player2(player_2_action_space) ## --> random
            # player_2_move = policy_player2(model2, action_space, current_state) # --> model itself
            current_state, reward, terminated, player_turn = env.step(player_2_move)

            current_state = func_modify(current_state).view(64).to(device)

        mean_loss = np.mean(loss_list)
        # print(f'Episode: {episode + 1}, Loss: {mean_loss}')
        overall_loss.append(mean_loss)
        if (episode + 1) % 100 == 0:
            print(f'episode: {episode + 1}, loss: {mean_loss}')
            model_path = f'C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/reinforcement_learning/Project/Phase_3_3D_Tic_Tac_Toe/models/ep_{episode + 1 + 6600}.pth'
            torch.save(model.state_dict(), model_path)
    return overall_loss

In [9]:
# Example usage
num_episodes = 1000  # Number of episodes for training
loss = train_td_model(model, num_episodes)  # Train the model


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
# Inference/Testing Against random policy
# Inference
def get_coordinates(position):
    x = int((position % 16) % 4)
    y = int((position % 16) / 4)
    z = int(position / 16)

    return x, y, z
def policy_player1(model, action_space, observation):
    value_dict = {}
    for action in action_space:
        x, y, z = get_coordinates(action)
        copy_tensor = func_modify(observation).detach().clone()
        copy_tensor[x, y, z] = 1
        value_dict[action] = model.forward(copy_tensor.view(4, 4, 4))
    action = max(value_dict, key = lambda k: value_dict[k])
    return action

def play_games(policy_player1, policy_player2, model, N = 100, render_mode = "computer"):
    wins = 0
    losses = 0
    draws = 0

    for i in tqdm(range(N)):
        env = tictactoe_4x4.TicTacToe4x4x4(render_mode)

        terminated = 0
        observation = [[[" " for _ in range(4)] for _ in range(4)] for _ in range(4)]
        reward = 0
        player_turn = "X"
        while not terminated:
            action_space = env.get_action_space()

            if player_turn == "X":
                action = policy_player1(model, action_space, observation)
            else:
                action = benchmark_policy_for_player2(action_space)

            observation, reward, terminated, player_turn = env.step(action)
        if reward == 1: wins += 1
        if reward == -1: losses += 1
        if reward == 0: draws += 1

    print(f'win percentage: {(wins / N) * 100}%')
    print(f'lose percentage: {(losses / N) * 100}%')
    print(f'draw percentage: {(draws / N) * 100}%')

new_model = MyNeuralNetwork()
state_new = torch.load('C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/reinforcement_learning/Project/Phase_3_3D_Tic_Tac_Toe/models/ep_8300.pth', map_location = torch.device(device))
new_model.load_state_dict(state_new)
play_games(policy_player1, benchmark_policy_for_player2, new_model)

  0%|          | 0/100 [00:00<?, ?it/s]

win percentage: 43.0%
lose percentage: 56.99999999999999%
draw percentage: 0.0%
